# Motion Transfer

Credits: https://github.com/iPERDance/iPERCore

In [1]:
# Install ffmpeg (ffprobe)
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [2]:
# set CUDA_HOME, here we use CUDA 10.1
import os
os.environ["CUDA_HOME"] = "/usr/local/cuda-10.1"

!echo $CUDA_HOME

/usr/local/cuda-10.1


## 1.1 Clone iPERCore Github Repo

In [3]:
!git clone https://github.com/iPERDance/iPERCore.git

Cloning into 'iPERCore'...
remote: Enumerating objects: 902, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 902 (delta 17), reused 29 (delta 14), pack-reused 864
Receiving objects: 100% (902/902), 23.05 MiB | 13.80 MiB/s, done.
Resolving deltas: 100% (334/334), done.


## 1.2 Setup iPERCore

In [ ]:
cd /content/iPERCore/

/content/iPERCore


In [ ]:
!python setup.py develop

In [ ]:
# Download all checkpoints

!wget -O assets/checkpoints.zip "http://101.32.75.151:10086/checkpoints.zip"
!unzip -o assets/checkpoints.zip -d assets/

!rm assets/checkpoints.zip

In [ ]:
# download samples
# !wget -O assets/samples.zip "https://1drv.ws/u/s!AjjUqiJZsj8whLobQPpoxo2hfhURrA?e=EUyIC2"
!wget -O assets/samples.zip  "http://101.32.75.151:12345/iper_plus_plus_latest_samples.zip"
!unzip -o assets/samples.zip -d  assets
!rm assets/samples.zip

In [ ]:
cd /content/iPERCore/

/content/iPERCore


In [ ]:
import os
import os.path as osp
import platform
import argparse
import time
import sys
import subprocess
from IPython.display import HTML
from base64 import b64encode

## Details of Config
 - gpu_ids (str): the gpu_ids, default is "0";
 - image_size (int): the image size, default is 512;
 - num_source (int): the number of source images for Attention, default is 2. Large needs more GPU memory;
 - assets_dir (str): the assets directory. This is very important, and there are the configurations and all pre-trained checkpoints;
 - output_dir (str): the output directory;

 - src_path (str): the source input information. 
       All source paths and it supports multiple paths, uses "|" as the separator between all paths. 
       The format is "src_path_1|src_path_2|src_path_3". 
       
       Each src_input is "path?=path1,name?=name1,bg_path?=bg_path1". 
       
       It must contain 'path'. If 'name' and 'bg_path' are empty, they will be ignored.

       The 'path' could be an image path, a path of a directory contains source images, and a video path.

       The 'name' is the rename of this source input, if it is empty, we will ignore it, and use the filename of the path.

       The 'bg_path' is the actual background path if provided, otherwise we will ignore it.
       
       There are several examples of formated source paths,

        1. "path?=path1,name?=name1,bg_path?=bg_path1|path?=path2,name?=name2,bg_path?=bg_path2",
        this input will be parsed as [{path: path1, name: name1, bg_path:bg_path1},
        {path: path2, name: name2, bg_path: bg_path2}];

        2. "path?=path1,name?=name1|path?=path2,name?=name2", this input will be parsed as
        [{path: path1, name:name1}, {path: path2, name: name2}];

        3. "path?=path1", this input will be parsed as [{path: path1}].

        4. "path1", this will be parsed as [{path: path1}].

 - ref_path (str): the reference input information.
       
       All reference paths. It supports multiple paths, and uses "|" as the separator between all paths.
       The format is "ref_path_1|ref_path_2|ref_path_3".

       Each ref_path is "path?=path1,name?=name1,audio?=audio_path1,fps?=30,pose_fc?=300,cam_fc?=150".

       It must contain 'path', and others could be empty, and they will be ignored.

       The 'path' could be an image path, a path of a directory contains images of a same person, and a video path.

       The 'name' is the rename of this source input, if it is empty, we will ignore it, and use the filename of the path.

       The 'audio' is the audio path, if it is empty, we will ignore it. If the 'path' is a video,
        you can ignore this, and we will firstly extract the audio information of this video (if it has audio channel).

       The 'fps' is fps of the final outputs, if it is empty, we will set it as the default fps 25.

       The 'pose_fc' is the smooth factor of the temporal poses. The smaller of this value, the smoother of the temporal poses. If it is empty, we will set it as the default 300. In the most cases, using the default 300 is enough, and if you find the poses of the outputs are not stable, you can decrease this value. Otherwise, if you find the poses of the outputs are over stable, you can increase this value.

       The 'cam_fc' is the smooth factor of the temporal cameras (locations in the image space). The smaller of this value, the smoother of the locations in sequences. If it is empty, we will set it as the default 150. In the most cases, the default 150 is enough.

       There are several examples of formated reference paths,

        1. "path?=path1,name?=name1,audio?=audio_path1,fps?=30,pose_fc?=300,cam_fc?=150|
            path?=path2,name?=name2,audio?=audio_path2,fps?=25,pose_fc?=450,cam_fc?=200",
            this input will be parsed as
            [{path: path1, name: name1, audio: audio_path1, fps: 30, pose_fc: 300, cam_fc: 150},
             {path: path2, name: name2, audio: audio_path2, fps: 25, pose_fc: 450, cam_fc: 200}]

        2. "path?=path1,name?=name1, pose_fc?=450|path?=path2,name?=name2", this input will be parsed as
        [{path: path1, name: name1, fps: 25, pose_fc: 450, cam_fc: 150},
         {path: path2, name: name2, fps: 25, pose_fc: 300, cam_fc: 150}].

        3. "path?=path1|path?=path2", this input will be parsed as
        [{path: path1, fps:25, pose_fc: 300, cam_fc: 150}, {path: path2, fps: 25, pose_fc: 300, cam_fc: 150}].

        4. "path1|path2", this input will be parsed as
        [{path: path1, fps:25, pose_fc: 300, cam_fc: 150}, {path: path2, fps: 25, pose_fc: 300, cam_fc: 150}].

        5. "path1", this will be parsed as [{path: path1, fps: 25, pose_fc: 300, cam_fc: 150}].

In [ ]:
# the gpu ids
gpu_ids = "0"

# the image size
image_size = 512

# the default number of source images, it will be updated if the actual number of sources <= num_source
num_source = 1

# the assets directory. This is very important, please download it from `one_drive_url` firstly.
assets_dir = "/content/iPERCore/assets"

# the output directory.
output_dir = "./results"

# the model id of this case. This is a random model name.
# model_id = "model_" + str(time.time())

# # This is a specific model name, and it will be used if you do not change it.
# model_id = "axing_1"

# symlink from the actual assets directory to this current directory
work_asserts_dir = os.path.join("./assets")
if not os.path.exists(work_asserts_dir):
    os.symlink(osp.abspath(assets_dir), osp.abspath(work_asserts_dir),
               target_is_directory=(platform.system() == "Windows"))

cfg_path = osp.join(work_asserts_dir, "configs", "deploy.toml")


In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "orange_justice"

# the source input information, here \" is escape character of double duote "
src_path = "\"path?=/content/iPERCore/assets/samples/sources/orange_justice\""



ref_path = "\"path?=/content/iPERCore/assets/samples/references/mabaoguo_short.mp4," \
             "name?=mabaoguo_short," \
             "pose_fc?=400\""

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

In [ ]:
# # This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
# model_id = "axing_1"

# # the source input information, here \" is escape character of double duote "
# src_path = "\"path?=/content/iPERCore/assets/samples/sources/axing_1,name?=axing_1\""


# ## the reference input information. There are three reference videos in this case.
# # here \" is escape character of double duote "
# ref_path = "\"path?=/content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4," \
#              "name?=bantangzhuyi_1," \
#              "pose_fc?=300\""

# print(ref_path)

# !python -m iPERCore.services.run_imitator  \
#   --gpu_ids     $gpu_ids       \
#   --num_source  $num_source    \
#   --image_size  $image_size    \
#   --output_dir  $output_dir    \
#   --model_id    $model_id      \
#   --cfg_path    $cfg_path      \
#   --src_path    $src_path      \
#   --ref_path    $ref_path

"path?=/content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4,name?=bantangzhuyi_1,pose_fc?=300"
ffprobe -show_entries stream=codec_type -of json /content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4 -loglevel error
ffmpeg -y -i /content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4 -ab 160k -ac 2 -ar 44100 -vn ./results/primitives/bantangzhuyi_1/processed/audio.mp3 -loglevel quiet
ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate /content/iPERCore/assets/samples/references/bantangzhuyi_1.mp4
	Pre-processing: start...
----------------------MetaProcess----------------------
meta_input:
	path: /content/iPERCore/assets/samples/sources/axing_1
	bg_path: 
	name: axing_1
primitives_dir: ./results/primitives/axing_1
processed_dir: ./results/primitives/axing_1/processed
vid_info_path: ./results/primitives/axing_1/processed/vid_info.pkl
-------------------------------------------------------
---------------------

In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "orange_justice"



src_path = "\"outputs\character_1\""



ref_path = "\"reference\video_1""

print(ref_path)

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path